Explorando uma IA Generativa em um Pipeline ETL com Python

CEP para endereço e coordenada API Geocoding

In [1]:
import pandas as pd
import requests
import json
import unidecode

EXTRACT

Carregando dados .csv , lembrando que a planilha que contém acentos

In [2]:
df=pd.read_csv("saudepac.csv", sep=';',encoding = "latin1")
locep = df['CEP'].tolist()
df.head()

,MUNICÍPIO,NOME,CEP
0,BELEM,URE MATERNO INFANTIL - UREMIA,66040-020
1,BELEM,UEPA-MARCO,66087-662
2,BELEM,URE DEMÉTRIO MEDRADO,66120-680
3,BELEM,URE DIPE,66113-240
4,BELEM,URE PRESIDENTE VARGAS,66017-000


TRANSFORM

Repositório API: https://brasilapi.com.br/docs

O BrasilAPI utiliza as coordenadas provindas do OpenStreetMap

In [3]:
cep_api_url='https://brasilapi.com.br/api/cep/v2'

In [4]:
def get_local(cep):
  response = requests.get(f'{cep_api_url}/{cep}')
  return response.json() if response.status_code == 200 else None

address=[]
for count in range(len(locep)):
  address.append(get_local(locep[count]))

Formando novas variáveis: endereços e coordenadas

In [5]:
geocode=[]

for count in range(len(locep)):
    if None==address[count]:
        geocode.append(None)
    else:
        tudo=(f"{address[count]['street']}, {address[count]['neighborhood']}, {address[count]['city']}-{address[count]['state']}")
        geocode.append(tudo)

In [6]:
coordenadas=[]

for count in range(len(locep)):
    if None==address[count]:
        coordenadas.append(None)
    elif {}==address[count]['location']['coordinates']:
        coordenadas.append(None)
    else:
        longlat=(f"{address[count]['location']['coordinates']['longitude']}, {address[count]['location']['coordinates']['latitude']}")
        coordenadas.append(longlat)

LOAD

Criar uma nova planilha adicionando novas colunas com as variáveis de endereço e coordenadas

In [7]:
df['ENDEREÇO']=geocode
df['COORDENADAS']=coordenadas
#print(df)

In [8]:
df.to_csv("saudePAatualizado.csv",sep=';',encoding = "latin1")